
# Tenseal Playground
So since this is not going to be a part of the git repository I'm not going to be formal in here. So lets go on and do the exploration.

## Should we use CKKS?
No since then we should handle the noise and then do the ZKP, hence it would lead to nasty stuff. So we should use the BFV scheme. Also the BFV scheme is faster and we can find the upper limit for our operations.

Since we are going to calculate the variance of the data we need to have:

$$ \sigma^2 = \sum_{i=1}^{n} x_i^2 - \frac{1}{n}(\sum_{i=1}^{n} x_i)^2 $$

So we need two things:

$$ \sum_{i=1}^{n} x_i^2 $$

$$ \sum_{i=1}^{n} x_i $$


Then the rest is easy. We can calculate the variance after the decryption.

We then need to add bias and random number to it. So we can do the ZKP.

Considering that we want to do the process for 100 meters of the street and the length of each car being 3.5 to 5 meters and also 0.5 meters for the space between the cars. We can say that we have at most 25 cars per each lane and considering at most four lanes the result would be 100 cars. So per each phase the maximum number we need to deal with is in terms of `cl` the car length, and `s` the space between the cars and `sl` street length is:

number of cars = street length / car length + space between cars)

Maximum number would be for the sum of the squares of the speeds with the bias and maximum random number possible. So the number would be:

max number  = number of cars * max speed^2 + bias + random number

Also the bias and the random number must be the same hence:

max number = number of cars * max speed^2 + 2 * bias

So we can calculate the upper limit for the operations.

max number = 100 * 100^2 +  2 * 10_000 = 1_020_000 

So this would be the plain text upper limit for the operations.

In [68]:
import tenseal as ts

context = ts.context(ts.SCHEME_TYPE.BFV, poly_modulus_degree=4096, plain_modulus=1032193)
context

Lets encrypt a list of numbers and then decrypt them.

In [69]:
plain = [1, 2, 3, 4, 5]
encrypted_vector = ts.bfv_vector(context, plain)
print("Encrypted vector size is: ", encrypted_vector.size())
print("Encrypted vector: ", encrypted_vector.decrypt())

Encrypted vector size is:  5
Encrypted vector:  [1, 2, 3, 4, 5]


Let's do serializations.

In [70]:
ev = encrypted_vector.serialize()
len(ev)

88531

In [71]:
pvc = context.serialize(save_secret_key=True)
print(f'Is context public: {context.is_public()}')
len(pvc)

Is context public: False


204439

In [72]:
context.make_context_public()
puc = context.serialize()
len(puc)

411077

Now lets deserialize stuff

In [73]:
context2 = ts.context_from(puc)
print(f'Is context public: {context2.is_public()}')
context2

Is context public: True


In [74]:
encrypted_vector2 = ts.bfv_vector_from(context2, ev)
print("Encrypted vector size is: ", encrypted_vector2.size())

Encrypted vector size is:  5


In [75]:
context3 = ts.context_from(pvc)
print(context3.is_public())

False


In [76]:
encrypted_vector3 = ts.bfv_vector_from(context3, ev)
print("Encrypted vector size is: ", encrypted_vector3.size())

Encrypted vector size is:  5


In [77]:
encrypted_vector3.decrypt()

[1, 2, 3, 4, 5]